In [227]:
import numpy as np
from functools import cache
from itertools import zip_longest
import sys

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [228]:
class Piece():
  rows: np.ndarray[np.uint8, np.uint8]
  columns: np.ndarray[np.uint8, np.uint8]
  rows_height: int
  rows_width: int
  cols_height: int
  cols_width: int

  def __init__(self, rows: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8], columns: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8]):
    rows = np.asarray(rows)
    columns = np.asarray(columns)

    # Trim piece to size
    while rows.size and ~np.any(rows[-1]) and columns.size and ~np.any(columns[-1]):
       rows = rows[:-1]
       columns = columns[:-1]

    while rows.size and ~np.any(rows[:,-1]) and columns.size and ~np.any(columns[:,-1]):
       rows = rows[:,:-1]
       columns = columns[:,:-1]
    
    # Edge case: when all rows or columns are trimmed from one side
    self.rows_height = 0 if not rows.size else rows.shape[0]
    self.rows_width = 0 if not rows.size else rows.shape[1]
    self.cols_height = 0 if not columns.size else columns.shape[0]
    self.cols_width = 0 if not columns.size else columns.shape[1]
    self.rows = np.packbits(rows, axis=0)
    self.columns = np.packbits(columns, axis=0)

  def __repr__(self):
    rows = np.unpackbits(self.rows, axis=0, count=self.rows_height)
    columns = np.unpackbits(self.columns, axis=0, count=self.cols_height)
    buffer = []
    for row, column in zip_longest(rows, columns):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else ' ' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else ' ' for elem in column))
    return "\n".join(buffer)
  
  def __hash__(self):
     return hash((tuple(self.rows.flatten()), tuple(self.columns.flatten())))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return np.array_equal(self.rows, other.rows) and np.array_equal(self.columns, other.columns)
      return NotImplemented
    
  def rotated(self):
    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.rows_height)
    unpacked_columns = np.unpackbits(self.columns, axis=0, count=self.cols_height)

    rows = np.rot90(unpacked_columns, 1)
    columns = np.rot90(unpacked_rows, 1)

    while rows.size and ~np.any(rows[0]) and columns.size and ~np.any(columns[0]):
      # top row is empty, so we can shift digit up
      rows = np.roll(rows, -1, 0)
      columns = np.roll(columns, -1, 0)

    return Piece(rows, columns)
  
  @cache
  def rotations(self):
    a = self
    b = a.rotated()
    c = b.rotated()
    d = c.rotated()
    return set((a, b, c, d))

pieces = [
  Piece(rows=((True, False), (True, False), (False, False)), columns=((True, True, False), (False, False, False))),
  Piece(rows=((False, False), (False, False), (False, False)), columns=((True, False, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((False, True, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((False, True, False), (False, True, False))),
  Piece(rows=((False, False), (True, False), (False, False)), columns=((True, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, False, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, False, False), (True, True, False))),
  Piece(rows=((True, False), (False, False), (False, False)), columns=((False, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, True, False), (True, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, True, False), (False, True, False))),
]

print(sys.getsizeof(pieces[0].rows))

129


In [229]:
pieces[0]

 -
| |
 -

In [230]:
pieces[1]

|
|

In [231]:
pieces[9].rotations()

{   -
 | | |
  - -,
  -
 |  
  -
 | |
  -,
  -
 | |
  -
   |
  -,
  - -
 | | |
  -  }

In [232]:
pieces[0].rotations()

{ -
 | |
  -}

In [233]:
class Puzzle():
  ROWS_HEIGHT = 5
  ROWS_WIDTH = 5
  COLS_HEIGHT = 4
  COLS_WIDTH = 6
  rows: np.ndarray[np.uint8, np.uint8]
  cols: np.ndarray[np.uint8, np.uint8]

  def __init__(
      self,
      rows=np.packbits(np.zeros((ROWS_HEIGHT,ROWS_WIDTH), dtype=np.uint8), axis=0),
      cols=np.packbits(np.zeros((COLS_HEIGHT,COLS_WIDTH), dtype=np.uint8), axis=0),
    ):
    # self.rows = np.packbits(rows, axis=0)
    # self.cols = np.packbits(cols, axis=0)
    self.rows = rows
    self.cols = cols

  def __repr__(self):
    rows = np.unpackbits(self.rows, axis=0, count=self.ROWS_HEIGHT)
    cols = np.unpackbits(self.cols, axis=0, count=self.COLS_HEIGHT)
    buffer = []
    for row, column in zip_longest(rows, cols):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else '.' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else '.' for elem in column))
    return "\n".join(buffer)

  def __hash__(self):
    return hash((tuple(self.rows.flatten()), tuple(self.cols.flatten())))

  def __eq__(self, other):
    if isinstance(other, Puzzle):
        return np.array_equal(self.rows, other.rows) and np.array_equal(self.cols, other.cols)
    return NotImplemented

  def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    if (toprow + piece.rows_height > self.ROWS_HEIGHT):
      return None
    if (topcol + piece.rows_width > self.ROWS_WIDTH):
      return None
    if (toprow + piece.cols_height > self.COLS_HEIGHT):
      return None
    if (topcol + piece.cols_width > self.COLS_WIDTH):
      return None
    
    mask_rows = np.zeros_like(self.rows)
    if piece.rows.size:
      mask_rows[:,topcol:topcol+piece.rows.shape[1]] = piece.rows >> toprow

    mask_cols = np.zeros_like(self.cols)
    if piece.columns.size:
      mask_cols[:,topcol:topcol+piece.columns.shape[1]] = piece.columns >> toprow

    if np.any(np.bitwise_and(self.rows, mask_rows)):
      return None
    if np.any(np.bitwise_and(self.cols, mask_cols)):
      return None

    np.bitwise_or(self.rows, mask_rows, out=mask_rows)
    np.bitwise_or(self.cols, mask_cols, out=mask_cols)

    return Puzzle(mask_rows, mask_cols)
  
  def spaces_left(self):
    unpacked_rows = np.unpackbits(self.rows, axis=0, count=self.ROWS_HEIGHT)
    unpacked_cols = np.unpackbits(self.cols, axis=0, count=self.COLS_HEIGHT)
    return np.size(unpacked_cols) - np.count_nonzero(unpacked_cols) + np.size(unpacked_rows) - np.count_nonzero(unpacked_rows)

In [234]:
%timeit Puzzle().with_piece(pieces[7], 2, 3)

37.9 µs ± 5.75 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [235]:
%lprun -f Puzzle.with_piece Puzzle().with_piece(pieces[7], 2, 3)

Timer unit: 1e-07 s

Total time: 0.0002198 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_16496\2132562078.py
Function: with_piece at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                             def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    39         1         39.0     39.0      1.8      if (toprow + piece.rows_height > self.ROWS_HEIGHT):
    40                                                 return None
    41         1         20.0     20.0      0.9      if (topcol + piece.rows_width > self.ROWS_WIDTH):
    42                                                 return None
    43         1         11.0     11.0      0.5      if (toprow + piece.cols_height > self.COLS_HEIGHT):
    44                                                 return None
    45         1         12.0     12.0      0.5      if (topcol + piece.cols_width > self.COLS_WIDTH):
    46                                      

In [236]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [237]:
Puzzle().spaces_left()

49

In [238]:
Puzzle().with_piece(pieces[0], 3, 4)

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . -
. . . . | |
 . . . . -

In [239]:
Puzzle().with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [240]:
Puzzle().with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [241]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [244]:
from copy import copy

bad_puzzles = set()

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check for solved
def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
  if puzzle in bad_puzzles:
    return None
  if puzzle.spaces_left() == 2:
    return puzzle, used_pieces
  
  candidates = []
  for i in range(len(pieces)):
    if i in used_pieces:
      continue

    for rotation in pieces[i].rotations():
      for row in range(0, puzzle.ROWS_HEIGHT):
        for col in range(0, puzzle.COLS_HEIGHT):
          candidate = puzzle.with_piece(rotation, row, col)
    
          if candidate is not None:
            if candidate in bad_puzzles:
              continue

            candidate_used_pieces = copy(used_pieces)
            candidate_used_pieces[i] = (rotation, row, col)
            candidates.append((candidate, candidate_used_pieces))
    
  for candidate, candidate_used_pieces in candidates:
    result = attempt(candidate, candidate_used_pieces)
    if result is not None:
      return result

  bad_puzzles.add(puzzle)
  return None

In [245]:
rot9 = pieces[9].rotated().rotated().rotated()
puzzle = Puzzle().with_piece(rot9, 1, 0)
# solution = attempt(puzzle, {9: (rot9, 1, 0)})
# solution
%lprun -f attempt attempt(puzzle, {9: (rot9, 1, 0)})

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 29.5767 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_16496\389121045.py
Function: attempt at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
    11      3260     349122.0    107.1      0.1    if puzzle in bad_puzzles:
    12                                               return None
    13      3260    1067169.0    327.4      0.4    if puzzle.spaces_left() == 2:
    14                                               return puzzle, used_pieces
    15                                             
    16      3260      17390.0      5.3      0.0    candidates = []
    17     35858     224911.0      6.3      0.1    for i in range(len(pieces)):
    18     32599     197758.0      6.1      0.1      if i in used_pieces:
    19     21909      71137.0      3.2      0.0        continue
    20                                     

In [246]:
len(bad_puzzles)

3253

In [247]:
next(iter(bad_puzzles))

 - . - - .
| | | | . .
 - - . . .
| | | | . .
 - - - - .
. . | | | |
 . - - . -
. . | | | .
 . . - - .